# Interfacing with R via rpy2

In [1]:
# !rm sequence.index 2>/dev/null
# !wget -nd http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/20130502.phase3.sequence.index -O sequence.index

The system cannot find the path specified.
--2023-07-01 03:33:02--  http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/20130502.phase3.sequence.index
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.140
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.140|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67057948 (64M)
Saving to: 'sequence.index'

     0K .......... .......... .......... .......... ..........  0%  449K 2m26s
    50K .......... .......... .......... .......... ..........  0% 20.0M 74s
   100K .......... .......... .......... .......... ..........  0% 3.28M 56s
   150K .......... .......... .......... .......... ..........  0% 6.75M 44s
   200K .......... .......... .......... .......... ..........  0% 7.11M 37s
   250K .......... .......... .......... .......... ..........  0% 15.9M 32s
   300K .......... .......... .......... .......... ..........  0% 11.7M 28s
   350K .......... ..........

In [1]:
import os
from IPython.display import Image
import rpy2.robjects as robjects
import rpy2.robjects.lib.ggplot2 as ggplot2
from rpy2.robjects.functions import SignatureTranslatedFunction
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects import local_converter

Note: The libraries above are not loading, may be due to the limited computational power of my system.

Thus, the code blocks below can't be run :(

In [ ]:
read_delim = robjects.r('read.delim')
seq_data = read_delim('sequence.index', header=True, tringsAsFactors=False)

In [ ]:
print('This dataframe has %d columns and %d rows' %
(seq_data.ncol, seq_data.nrow))
print(seq_data.colnames)

In [ ]:
my_cols = robjects.r.ncol(seq_data)
print(my_cols)

In [ ]:
as_integer = robjects.r('as.integer')
match = robjects.r.match

my_col = match('READ_COUNT', seq_data.colnames)[0] # vector returned
print('Type of read count before as.integer: %s' % seq_data[my_col - 1].rclass[0])
seq_data[my_col - 1] = as_integer(seq_data[my_col - 1])
print('Type of read count after as.integer: %s' % seq_data[my_col - 1].rclass[0])

In [ ]:
robjects.r.assign('seq.data', seq_data)

In [ ]:
from rpy2.robjects.functions import SignatureTranslatedFunction

ggplot2.theme = SignatureTranslatedFunction(ggplot2.theme, init_prm_translate = {'axis_text_x': 'axis.text.x'})
bar = ggplot2.ggplot(seq_data) + ggplot2.geom_bar() + ggplot2.aes_string(x='CENTER_NAME') + ggplot2.theme(axis_text_x=ggplot2.element_text(angle=90, hjust=1))
robjects.r.png('out.png', type='cairo-png')
bar.plot()
dev_off = robjects.r('dev.off')
dev_off()

In [ ]:
Image(filename='out.png')

In [ ]:
robjects.r('yri_ceu <- seq.data[seq.data$POPULATION %in% c("YRI", "CEU") & seq.data$BASE_COUNT < 2E9 & seq.data$READ_COUNT < 3E7, ]')
yri_ceu = robjects.r('yri_ceu')

In [ ]:
scatter = ggplot2.ggplot(yri_ceu) + ggplot2.aes_string(x='BASE_COUNT', y='READ_COUNT',shape='factor(POPULATION)', col='factor(ANALYSIS_GROUP)')
+ ggplot2.geom_point()
robjects.r.png('out.png')
scatter.plot()

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
with localconverter(ro.default_converter + pandas2ri.converter):
    pd_yri_ceu = ro.conversion.rpy2py(yri_ceu)
    del pd_yri_ceu['PAIRED_FASTQ']
with localconverter(ro.default_converter + pandas2ri.converter):
    no_paired = ro.conversion.py2rpy(pd_yri_ceu)
robjects.r.assign('no.paired', no_paired)
robjects.r("print(colnames(no.paired))")

# Finish!